In [2]:
import os, sys
sys.path.insert(0, os.path.abspath('./mds'))



In [3]:
from typing import List, Literal, Tuple
from pydantic import EmailStr, Extra
from mds.utils import FairscapeBaseModel, OrganizationCompactView, ProjectCompactView, DatasetCompactView, SoftwareCompactView, ComputationCompactView
import pymongo

In [5]:

class User(FairscapeBaseModel, extra=Extra.allow):
    type: Literal["Person"]
    email: EmailStr  # requires installation of module email-validator
    password: str
    is_admin: bool
    organizations: List[OrganizationCompactView]
    projects: List[ProjectCompactView]
    datasets: List[DatasetCompactView]
    software: List[SoftwareCompactView]
    computations: List[ComputationCompactView]

    def create(self, MongoCollection: pymongo.collection.Collection) -> Tuple[bool, str, int]: 

        # creating a new user we must set their owned objects to none
        self.projects = []
        self.datasets = []
        self.software = []
        self.computations = []

        try: 
            create_request = MongoCollection.insert_one(self.json(by_alias=True))

            if create_request.acknowledged:
                return (True, "", 200)
            else:
                return (False, "", 200)

        except pymongo.errors.ConnectionError as e:
            return (False, f"MongoConnectionError: {str(e)}", 500)
        except * as e:
            return (False, f"Error: {str(e)}", 400)


    def read(self, MongoCollection) -> Tuple[bool, str, int]:

        try: 
            query = test_collection.find_one(
                {"@id": test_data["@id"]}, 
                projection={"_id": False}
                )
            
        except pymongo.errors.ConnectionFailure as e:
            return (False, f"MongoConnectionError: {str(e)}", 500)
        except * as e:
            return (False, f"Undetermined Error: {e}", 400)
        else:

            # check that the results are not empty
            if query:
                # update class with values from database
                for key, value in query.items():
                    setattr(self, key, value)

                return (True, "", 200)
            else:
                return (False, "No record found", 404)



SyntaxError: invalid syntax (Temp/ipykernel_16272/1054831783.py, line 30)

In [1]:
import pymongo

In [4]:
test_user = User(**test_data)

NameError: name 'test_data' is not defined

In [4]:
# create a mongo connection for testing
from mds.mongo import MongoConfig

In [5]:
db_config = MongoConfig(
	host_uri = "localhost",
	port = 27017,
	user = "root",
	password = "example",
	database = "test"
)

In [6]:
db_config.

AttributeError: 'MongoConfig' object has no attribute 'test'

In [49]:
insert_result = test_collection.insert_one(
	test_user.dict(by_alias=True)
)

In [50]:
insert_result.acknowledged	

True

In [51]:
insert_result.inserted_id

ObjectId('622a5f1ba6034953d541e6e1')

In [52]:
test_user.json(by_alias=True)

'{"@id": "ark:99999/test-user", "@type": "Person", "name": "Test User", "email": "test@example.org", "password": "test", "is_admin": false, "organizations": [], "projects": [], "datasets": [], "software": [], "computations": []}'

In [58]:
# when given an @id

class TestUserCRUD(unnittest.TestCase):
	user_data = {
		"@id": "ark:99999/test-user",
		"name": "Test User",
		"type": "Person",
		"email": "test@example.org",
		"password": "test",
		"is_admin": False,
		"organizations": [],
		"datasets": [],
		"projects": [],
		"software": [],
		"computations": []
		}		


	def setUp(self):
		db_config = MongoConfig(
			host_uri = "localhost",
			port = 27017,
			user = "root",
			password = "example",
			database = "test"
			)
		mongo_client = db_config.connect()
		test_db = mongo_client["test"]
		self.test_collection = test_db["test"]


	def test_user_create(self):
		test_user = User(**self.user_data)
		test_user.create(self.test_collection)


	def test_user_read(self):
		test_user = User.construct(
			_fields_set={"id"}, 
			**{"@id": test_data["@id"]}
			)
		
		test_user.read(self.test_collection)


In [60]:
read_test

User(@id='ark:99999/test-user', @type='Person', name='Test User', email='test@example.org', password='test', is_admin=False, organizations=[], projects=[], datasets=[], software=[], computations=[])

In [32]:
# get one example

User(**test_collection.find_one({"@id": test_data["@id"]}, projection={"_id": False}))

User(id='ark:99999/test-user', type='Person', name='Test User', email='test@example.org', password='test', is_admin=False, organizations=[], projects=[], datasets=[], software=[], computations=[])

In [ ]:
# update a document


# find the rest 


test_collection.find_one_and_update()

In [ ]:
# delete operation


In [7]:
# create indexes for unique @id
db_config = MongoConfig(
	host_uri = "localhost",
	port = 27017,
	user = "root",
	password = "example",
	database = "test"
	)
mongo_client = db_config.connect()
test_db = mongo_client["test"]
test_collection = test_db["test"]




In [10]:
user_data = {
	"@id": "ark:99999/test-user",
	"name": "Test User",
	"type": "Person",
	"email": "test@example.org",
	"password": "test",
	"is_admin": False,
	"organizations": [],
	"datasets": [],
	"projects": [],
	"software": [],
	"computations": []
	}	

In [15]:
if test_collection.find_one({"@id": user_data["@id"]}):
	print("found")

found


In [ ]:

test_collection.create_index("@id", unique=True)
test_collection.create_index("@type")
test_collection.create_index("owner")